# prerun

In [1]:
!pip install leveldb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.5/236.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for leveldb: filename=leveldb-0.201-cp311-cp311-linux_x86_64.whl size=2049019 sha256=0a7b1c8ccda322e83257fe987f3ef868e60adbc216afe8bf351697a8f50e084f
  Stored in directory: /root/.cache/pip/wheels/ae/bc/56/d98f2bd61408b45fb7504f8689809603ca751cd81b017436e9
Successfully built leveldb


# engine

In [2]:
import os
import zipfile
import json
import csv
import glob
import struct
import leveldb
from datetime import datetime

def extract_zip(zip_path, extract_path):
    os.makedirs(extract_path, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"✅ Extraction complete! Files are saved in: {extract_path}")

def decode_varint(data):
    shift = 0
    result = 0
    for i, byte in enumerate(data):
        result |= (byte & 0x7F) << shift
        if not (byte & 0x80):
            return result, data[i+1:]
        shift += 7
    return result, data

def parse_indexeddb_key(key):
    if not key:
        return "Empty Key"
    key_type = key[0]
    try:
        if key_type == 0x00:
            return f"DatabaseMetadata: {key[1:].hex()}"
        elif key_type == 0x01:
            object_store_id, rest = decode_varint(key[1:])
            return f"ObjectStore({object_store_id}): {rest.hex()}"
        elif key_type == 0x02:
            object_store_id, rest = decode_varint(key[1:])
            index_id, rest = decode_varint(rest)
            return f"Index({object_store_id}, {index_id}): {rest.hex()}"
        elif key_type == 0x03:
            object_store_id, rest = decode_varint(key[1:])
            index_id, rest = decode_varint(rest)
            primary_key, rest = decode_varint(rest)
            return f"IndexKey({object_store_id}, {index_id}, {primary_key}): {rest.hex()}"
        else:
            return f"UnknownKey: {key.hex()}"
    except Exception as e:
        return f"Failed to parse key: {e}"

def scan_leveldb(ldb_path, output_file="indexeddb_dump.json"):
    if not os.path.exists(ldb_path):
        print(f"Error: LevelDB path '{ldb_path}' not found!")
        return
    db = leveldb.LevelDB(ldb_path)
    data = []
    for key, value in db.RangeIter():
        entry = {
            "ParsedKey": parse_indexeddb_key(key),
            "RawKey": key.hex(),
            "Value": value.decode('utf-8', errors='ignore') if value else "EMPTY"
        }
        data.append(entry)
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"✅ LevelDB scan complete! Data saved to: {output_file}")

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Extract all account entries (accounts do not have an "actionType")
def extract_all_accounts(data):
    accounts = {}
    for entry in data:
        if "Value" in entry:
            try:
                value = json.loads(entry["Value"])
                if isinstance(value, dict) and "id" in value and "actionType" not in value:
                    accounts[str(value["id"])] = value
            except json.JSONDecodeError:
                continue
    return accounts

# Process actions and collect mentioned account IDs from logs (using "itemIds")
def process_actions(data):
    actions = []
    mentioned_account_ids = set()
    for entry in data:
        if "Value" in entry:
            try:
                value = json.loads(entry["Value"])
                if isinstance(value, dict) and "actionType" in value:
                    actions.append(value)
                    if "logs" in value:
                        for log in value["logs"]:
                            if "itemIds" in log and isinstance(log["itemIds"], list):
                                for account_id in log["itemIds"]:
                                    mentioned_account_ids.add(str(account_id))
            except json.JSONDecodeError:
                continue
    return actions, mentioned_account_ids

def write_action_to_csv(action, index, all_accounts):
    # Create a CSV file for the action
    filename = f"action_{index}_{action.get('actionType', 'unknown')}.csv"
    account_count = 0
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Action Type", "Target", "Updated At", "UUID", "Username", "Full Name", "ID", "Is Private", "Is Verified", "Profile Pic URL", "Requested By Viewer", "Status"])
        writer.writerow([
            action.get("actionType", ""),
            action.get("target", ""),
            action.get("updatedAt", ""),
            action.get("uuid", ""),
            "", "", "", "", "", "", ""
        ])
        if "logs" in action:
            for log in action["logs"]:
                if "itemIds" in log and isinstance(log["itemIds"], list):
                    for account_id in log["itemIds"]:
                        aid = str(account_id)
                        account = all_accounts.get(aid)
                        if account:
                            writer.writerow([
                                "", "", "", "",
                                account.get("username", ""),
                                account.get("full_name", ""),
                                account.get("id", ""),
                                account.get("is_private", ""),
                                account.get("is_verified", ""),
                                account.get("profile_pic_url", ""),
                                account.get("requested_by_viewer", ""),
                                account.get("status", "")
                            ])
                        else:
                            writer.writerow(["", "", "", "", "", "", aid, "", "", "", "", ""])
                        account_count += 1
    return filename, account_count

def write_remaining_accounts_to_csv(remaining_accounts):
    filename = "remained.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Username", "Full Name", "ID", "Is Private", "Is Verified", "Profile Pic URL", "Requested By Viewer", "Status"])
        for account in remaining_accounts.values():
            writer.writerow([
                account.get("username", ""),
                account.get("full_name", ""),
                account.get("id", ""),
                account.get("is_private", ""),
                account.get("is_verified", ""),
                account.get("profile_pic_url", ""),
                account.get("requested_by_viewer", ""),
                account.get("status", "")
            ])
    return filename

def create_output_zip(zip_filename="output.zip"):
    # List of files to include
    files_to_zip = []
    # Include the JSON dump if it exists
    if os.path.exists("indexeddb_dump.json"):
        files_to_zip.append("indexeddb_dump.json")
    # Include all action CSV files
    files_to_zip.extend(glob.glob("action_*.csv"))
    # Include the remained CSV file if exists
    if os.path.exists("remained.csv"):
        files_to_zip.append("remained.csv")

    # Create zip archive and add files
    with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
        for f in files_to_zip:
            zipf.write(f)
    print(f"✅ All output files have been zipped into: {zip_filename}")



# run

In [3]:

def main():
    # Step 1: Extract the provided zip file
    zip_path = "h.zip"                # Your input zip file
    extract_path = "./extracted"        # Extraction folder
    folder_name = "h"




    #------------------------
    extract_zip(zip_path, extract_path)

    # Step 2: Scan LevelDB from the extracted folder and create JSON dump
    ldb_path = os.path.join(extract_path, folder_name)
    scan_leveldb(ldb_path)

    # Step 3: Process the JSON dump to extract accounts and actions
    data = load_json("indexeddb_dump.json")
    all_accounts = extract_all_accounts(data)
    actions, mentioned_account_ids = process_actions(data)

    # Step 4: Write CSV files for each action
    total_action_accounts = 0
    for i, action in enumerate(actions, start=1):
        csv_file, count = write_action_to_csv(action, i, all_accounts)
        print(f"CSV file created: {csv_file} | Accounts in action: {count}")
        total_action_accounts += count

    # Step 5: Write CSV for remaining accounts not mentioned in any action
    remaining_accounts = {aid: acct for aid, acct in all_accounts.items() if aid not in mentioned_account_ids}
    remained_csv = write_remaining_accounts_to_csv(remaining_accounts)

    # Step 6: Print analysis
    total_mentioned_ids = sum(len(log.get("itemIds", [])) for action in actions for log in action.get("logs", []))
    print("\n--- Analysis ---")
    print(f"Total actions found: {len(actions)}")
    print(f"Total accounts found: {len(all_accounts)}")
    print(f"Total accounts mentioned in actions (from logs): {total_mentioned_ids}")
    print(f"Total accounts in actions (written to CSV): {total_action_accounts}")
    print(f"Remaining accounts not mentioned in any action: {len(remaining_accounts)}")
    print(f"Total accounts overall: {total_action_accounts + len(remaining_accounts)}")
    print(f"CSV file created for remaining accounts: {remained_csv}")

    # Step 7: Zip all output files
    create_output_zip()

if __name__ == "__main__":
    main()

✅ Extraction complete! Files are saved in: ./extracted
✅ LevelDB scan complete! Data saved to: indexeddb_dump.json
CSV file created: action_1_unfollow.csv | Accounts in action: 207
CSV file created: action_2_unfollow.csv | Accounts in action: 0
CSV file created: action_3_follow.csv | Accounts in action: 60
CSV file created: action_4_unfollow.csv | Accounts in action: 0
CSV file created: action_5_unfollow.csv | Accounts in action: 0
CSV file created: action_6_unfollow.csv | Accounts in action: 0
CSV file created: action_7_unfollow.csv | Accounts in action: 1200
CSV file created: action_8_unfollow.csv | Accounts in action: 1418
CSV file created: action_9_unfollow.csv | Accounts in action: 1407
CSV file created: action_10_follow.csv | Accounts in action: 74
CSV file created: action_11_unfollow.csv | Accounts in action: 0
CSV file created: action_12_unfollow.csv | Accounts in action: 0
CSV file created: action_13_unfollow.csv | Accounts in action: 0
CSV file created: action_14_unfollow.csv